# 第13章 ニューラルネットワーク

- 著者オリジナル: https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch13/ch13.ipynb
- Theano を使って最適化された機械学習コードを作成する
  - http://deeplearning.net/software/theano/
- 人工ニューラルネットワークの活性化関数を選択する
- すばやく簡単に実験を行うためにディープラーニングライブラリ Keras を使用する
  - https://keras.io/ , https://keras.io/ja/
  
## 13.1 Theano を使った式の構築、コンパイル、実行

### 13.1.1 Theano とは何か

### 13.1.2 はじめてのTheano

```
$ pip install Theano
```

In [1]:
import theano
from theano import tensor as T

# 初期化: scalar メソッドではスカラー(単純な配列)を生成
x1 = T.scalar()
w1 = T.scalar()
w0 = T.scalar()
z1 = w1 * x1 + w0

# コンパイル
net_input = theano.function(inputs=[w1, x1, w0], outputs=z1)

# 実行
net_input(2.0, 1.0, 0.5)

array(2.5)

### 13.1.3 Theano を設定する

In [9]:
# 浮動小数点数の型のデフォルトを確認
print(theano.config.floatX)

# 浮動小数点数の型を float32 に設定
# GPU の場合は float32 にしないといけないらしい
theano.config.floatX = 'float32'

# CPU と GPU どっちを使うかの設定を確認
print(theano.config.device)

float32
cpu


- 環境変数で設定の変更が可能

```
export THEANO_FLAGS=floatX=float32
```

- cpuを使って計算する場合

```
THEANO_FLAGS=device=cpu,floatX=float64 python <pythonスクリプト>
```

- gpuを使う場合

```
THEANO_FLAGS=device=gpu,floatX=float32 python <pythonスクリプト>
```

- ~/.theanorc ファイルにデフォルト設定を書くことも可能

```
[global]
floatX=float32
device-gpu
```

### 13.1.4 配列構造を操作する

In [12]:
import numpy as np

# 初期化
# Theanoを64ビットモードで実行している場合は、fmatrixの代わりにdmatrixを使用する必要がある
x = T.fmatrix(name='x')
x_sum = T.sum(x, axis=0)

# コンパイル
calc_sum = theano.function(inputs=[x], outputs=x_sum)

# 実行(Pythonリスト)
ary = [[1, 2, 3], [1, 2, 3]]
print('Column sum:', calc_sum(ary))

# 実行(Numpy配列)
ary = np.array([[1, 2, 3], [1, 2, 3]], dtype=theano.config.floatX)
print('Column sum:', calc_sum(ary))

Column sum: [ 2.  4.  6.]
Column sum: [ 2.  4.  6.]


In [15]:
import theano
from theano import tensor as T

# 初期化
x = T.fmatrix('x')
w = theano.shared(np.asarray([[0.0, 0.0, 0.0]], dtype=theano.config.floatX))
z = x.dot(w.T)
update = [[w, w + 1.0]]

# コンパイル
net_input = theano.function(inputs=[x], updates=update, outputs=z)

# 実行
data = np.array([[1, 2, 3]], dtype=theano.config.floatX)
for i in range(5):
    print('z{}:'.format(i), net_input(data))

z0: [[ 0.]]
z1: [[ 6.]]
z2: [[ 12.]]
z3: [[ 18.]]
z4: [[ 24.]]
